# Week 3: Prompt Engineering Best Practices

- Topics: Advanced prompt engineering techniques, designing effective prompts, common pitfalls, and improvement strategies.
- Hands-on: Experimenting with prompts to achieve desired responses.

## Introduction

In recent years, **prompt engineering** has emerged as a key process in the effective use of large language models. As Phoenix and Taylor (2024) describe it:

> "Prompt engineering is the process of discovering prompts that reliably yield useful or desired results."

This iterative approach focuses on refining prompts to optimize interactions with generative AI systems, achieving consistent and meaningful outputs (Phoenix and Taylor, 2024).


## Definitions of Prompt Engineering

To gain a comprehensive understanding of prompt engineering, here are several authoritative definitions from recent literature:

- **Phoenix and Taylor (2024):**
  
  "Prompt engineering is the process of discovering prompts that reliably yield useful or desired results."
  
  *(Phoenix and Taylor, 2024)*

- **Liu et al. (2023):**

  "Prompt engineering involves crafting input prompts that guide pre-trained language models to produce desired outputs, effectively leveraging their knowledge without the need for fine-tuning."

  *(Liu et al., 2023)*

- **Brown et al. (2020):**

  "Prompt engineering is the process of designing task-specific prompts to elicit desired behavior from language models, enabling them to perform a wide range of tasks without explicit training."

  *(Brown et al., 2020)*

- **Reynolds and McDonell (2021):**

  "Prompt engineering is the art of designing prompts that effectively communicate the user's intent to a language model, facilitating accurate and relevant responses."

  *(Reynolds and McDonell, 2021)*

- **White et al. (2023):**

  "Prompt engineering is a methodology for structuring inputs to large language models to achieve desired outcomes, often involving iterative refinement and understanding of model behavior."

  *(White et al., 2023)*


In [15]:
import os
import openai
from anthropic import Anthropic 
from dotenv import load_dotenv
from langchain_anthropic import ChatAnthropic

# Load environment variables from .env file
load_dotenv()

# Function to load environment variables or raise an error if not found
def get_env_var(var: str):
    value = os.getenv(var)
    if value is None:
        raise ValueError(f"{var} not found in environment variables. Make sure it is set in your .env file.")
    return value

# Load API keys from the environment
langchain_api_key = get_env_var("LANGCHAIN_API_KEY")
langchain_tracing_v2 = get_env_var("LANGCHAIN_TRACING_V2")
openai_api_key = get_env_var("OPENAI_API_KEY")
tavily_api_key = get_env_var("TAVILY_API_KEY")
anthropic_api_key = get_env_var("ANTHROPIC_API_KEY")
grok_api_key = get_env_var("GROK_API_KEY")

# Set up OpenAI models
from langchain_openai import ChatOpenAI
gpt4o_chat = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=openai_api_key)
gpt35_chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0, openai_api_key=openai_api_key)

# Direct Anthropic client
claude = Anthropic(api_key=anthropic_api_key)

# LangChain integration
claude_chat = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0, anthropic_api_key=anthropic_api_key)

In [16]:
import os
import requests
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Load the Grok API key from environment variables
grok_api_key = os.getenv("GROK_API_KEY")

# Define the Grok API endpoint and headers
url = "https://api.x.ai/v1/chat/completions"
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {grok_api_key}"
}

# Define the payload as shown in the example
payload = {
    "messages": [
        {
            "role": "system",
            "content": "You are Grok, a chatbot inspired by the Hitchhikers Guide to the Galaxy."
        },
        {
            "role": "user",
            "content": "What is the answer to life and universe?"
        }
    ],
    "model": "grok-beta",
    "stream": False,
    "temperature": 0
}

# Send the request to the Grok API
try:
    response = requests.post(url, headers=headers, json=payload)
    response.raise_for_status()  # Raise an error if the request fails
    print("Response:", response.json())
except requests.exceptions.RequestException as e:
    print("Error:", e)


Response: {'id': '8289e74c-f380-4bb8-a123-431990509628', 'object': 'chat.completion', 'created': 1731499306, 'model': 'grok-beta', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'The answer to life, the universe, and everything is **42**. However, this answer was famously provided by Deep Thought, the supercomputer from Douglas Adams\' "The Hitchhiker\'s Guide to the Galaxy," and it\'s meant to be humorous and thought-provoking rather than a literal solution. The real fun comes from pondering what the question might be, as the characters in the story realize they don\'t actually know the ultimate question to which 42 is the answer. So, in a way, the search for meaning continues, both in the story and in our lives, often with a humorous twist.', 'refusal': None}, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 37, 'completion_tokens': 124, 'total_tokens': 161}, 'system_fingerprint': 'fp_14b89b2dfc'}


## We can then use Claude in two ways:

1. Direct Anthropic client:

In [2]:
response = claude.messages.create(
    model="claude-3-5-sonnet-20240620",  # Using base model identifier
    max_tokens=1024,
    messages=[{"role": "user", "content": "Hello!"}]
)
print(response.content)

[TextBlock(text='Hello! How can I assist you today? Feel free to ask me any questions or let me know if you need help with anything.', type='text')]


2. LangChain integration (similar to how we're using OpenAI):

In [3]:
response = claude_chat.invoke("Hello!")
print(response.content)

Hello! How can I assist you today? Feel free to ask me any questions or let me know if you need help with anything.


In [17]:
def compare_responses(prompt: str, include_claude=True, include_gpt4=True, include_gpt35=True):
    """
    Compare responses from different models for the same prompt.
    """
    results = {}

    if include_claude:
        claude_response = claude_chat.invoke(prompt)
        results["Claude-3.5-Sonnet"] = claude_response.content

    if include_gpt4:
        gpt4_response = gpt4o_chat.invoke(prompt)
        results["GPT-4o"] = gpt4_response.content

    if include_gpt35:
        gpt35_response = gpt35_chat.invoke(prompt)
        results["GPT-3.5"] = gpt35_response.content

    # Print results in a formatted way
    print(f"\nPrompt: {prompt}\n")
    print("-" * 80)
    for model, response in results.items():
        print(f"\n{model}:\n")
        print(response)
        print("-" * 80)

# Let's test with some example prompts
test_prompts = [
    "Generate a list of creative product names for a new eco-friendly cleaning brand."
]

# Run comparisons for each prompt
for prompt in test_prompts:
    compare_responses(prompt)



Prompt: Generate a list of creative product names for a new eco-friendly cleaning brand.

--------------------------------------------------------------------------------

Claude-3.5-Sonnet:

Here's a list of creative product names for a new eco-friendly cleaning brand:

1. GreenSweep
2. EcoShine
3. NatureBright
4. PurifyEarth
5. CleanCanvas
6. BioBliss
7. EcoGlow
8. TerraSpark
9. PristinePlanet
10. VerdantVibe
11. AquaPure
12. EcoLuster
13. GaiaGleam
14. ZenClean
15. EarthErase
16. BotaniBlast
17. CrystalClear
18. HarmonyHouse
19. NectarNeat
20. OzoneOasis
21. EcoEssence
22. PurePolish
23. GreenGuard
24. BioBubble
25. EarthEmber
26. FreshFusion
27. NatureNurture
28. EcoElixir
29. PurePixie
30. TerraTidy
31. AquaAura
32. EcoEnergy
33. GreenGenie
34. BioBoost
35. EarthEcho
36. CleanConscious
37. NaturalNimbus
38. PurePetal
39. EcoEden
40. VitalVapor

These names combine eco-friendly themes, natural elements, and cleaning concepts to create unique and memorable brand identities.
-------

## Overview of the Five Principles to Improve the Prompt (Phoenix and Taylor, 2024)

The Five Principles of Prompting are as follows:

### Give Direction
Describe the desired style in detail, or reference a relevant persona

### Specify Format
Define what rules to follow, and the required structure of the response

### Provide Examples
Insert a diverse set of test cases where the task was done correctly

### Evaluate Quality
Identify errors and rate responses, testing what drives performance.

### Divide Labor
Split tasks into multiple steps, chained together for complex goals

These principles are not short-lived tips or hacks but are generally accepted conventions that are useful for working with any level of intelligence, biological or artificial. These principles are model-agnostic and should work to improve your prompt no matter which generative text or image model you’re using. We first published these principles in July 2022 in the blog post [“Prompt Engineering: From Words to Art and Copy”](https://oreil.ly/RYYiV), and they have stood the test of time, including mapping quite closely to OpenAI’s own [Prompt Engineering Guide](https://oreil.ly/dF8q-), which came a year later. Anyone who works closely with generative AI models is likely to converge on a similar set of strategies for solving common issues, and throughout this book you’ll see hundreds of demonstrative examples of how they can be useful for improving your prompts.

Popular Udemy course [The Complete Prompt Engineering for AI Bootcamp (70,000+ students)](https://oreil.ly/V40zg), which was based on the same principles.

Here’s a simple example of a prompt we could use as a starting point:

> **Simple Prompt**: “Generate a list of creative product names for a new eco-friendly cleaning brand.”

Using this prompt, the AI might return names like *EcoShine*, *GreenClean*, or *NatureBright*. However, based on the principles from Phoenix and Taylor’s (2024) book, this prompt could be refined to produce more consistent and higher-quality results.

---

### Five Principles to Improve the Prompt (Phoenix and Taylor, 2024)

1. **Provide Clear and Specific Direction**:
   - **Issue**: The simple prompt is vague about the style, tone, or specific attributes the product name should reflect.
   - **Improvement**: You could specify whether you want a single word or compound names, whether the names should be in plain English, or if they can include newly coined terms.
   - **Refined Prompt Example**: “Generate a list of catchy, eco-friendly product names that evoke freshness and cleanliness. Names should be easy to pronounce, memorable, and consist of one or two words in English.”

2. **Format the Output**:
   - **Issue**: The unformatted output might be inconsistent or hard to parse programmatically.
   - **Improvement**: Request a specific format, such as a numbered list or JSON format, to ensure predictability.
   - **Refined Prompt Example**: “Generate a numbered list of 5 eco-friendly product names for a cleaning brand. Each name should be in its own line.”

3. **Provide Examples**:
   - **Issue**: Without examples, the AI is basing its suggestions solely on its general training data, which may not align with the desired tone or style.
   - **Improvement**: Include examples of existing product names that fit your criteria to give the AI a clearer understanding of the desired output.
   - **Refined Prompt Example**: “Generate a list of eco-friendly product names similar in style to ‘EcoFresh,’ ‘GreenWave,’ and ‘PureClean.’ Each name should reflect qualities of sustainability and effectiveness.”

4. **Establish Evaluation Criteria**:
   - **Issue**: There’s no objective way to determine whether the output names meet the desired criteria.
   - **Improvement**: Define specific characteristics or ratings criteria to assess the quality of generated names.
   - **Refined Prompt Example**: “Generate a list of 5 eco-friendly product names that are short, memorable, and unique. The names should ideally not be found in a simple Google search for existing cleaning products.”

5. **Divide Complex Tasks into Subtasks**:
   - **Issue**: The prompt tries to handle multiple aspects of naming in one go, potentially leading to suboptimal results.
   - **Improvement**: Break down the task by asking the AI for individual qualities or styles first, then combine them.
   - **Refined Prompt Example**: “First, generate a list of adjectives associated with eco-friendliness (e.g., ‘Green,’ ‘Pure,’ ‘Eco’). Next, generate a list of words related to cleaning. Finally, combine these words to create a list of 5 unique product names.”

---

These principles offer a systematic approach to refining your prompt for production use, aligning with Phoenix and Taylor’s advice on cost-effectiveness and reducing iterative corrections (Phoenix and Taylor, 2024).

## References

- Phoenix, J., and Taylor, M. 2024. *Prompt Engineering for Generative AI*. Sebastopol, CA: Saxifrage, LLC, Just Understanding Data LTD, & O'Reilly Media, Inc. Available at: [O'Reilly Media](https://www.oreilly.com/library/view/prompt-engineering-for/9781098153427/)

- Brown, T. B., Mann, B., Ryder, N., Subbiah, M., Kaplan, J., Dhariwal, P., et al. 2020. Language Models are Few-Shot Learners. In *Advances in Neural Information Processing Systems*, 33, 1877–1901. Available at: [NeurIPS Proceedings](https://proceedings.neurips.cc/paper/2020/file/1457c0d6bfcb4967418bfb8ac142f64a-Paper.pdf)

- Liu, P., Yuan, W., Fu, J., Jiang, Z., Hayashi, H., and Neubig, G. 2023. Pre-train, Prompt, and Predict: A Systematic Survey of Prompting Methods in Natural Language Processing. *Trans. Assoc. Comput. Linguistics*. 11, 630–676. Available at: [ACL Anthology](https://aclanthology.org/2023.tacl-1.88.pdf)

- Reynolds, L., and McDonell, K. 2021. Prompt Programming for Large Language Models: Beyond the Few-Shot Paradigm. *arXiv preprint arXiv:2102.07350*. Available at: [arXiv](https://arxiv.org/abs/2102.07350)

- White, J., Narayanan, A., Chiu, S., Huang, Y., and Schmidt, D. 2023. A Comprehensive Survey on Prompt Engineering for Large Language Models. *arXiv preprint arXiv:2302.07842*. Available at: [arXiv](https://arxiv.org/abs/2302.07842)



